# Regresiones no lineales

Muchas relaciones en la realidad son no lineales. Por ejemplo, la utilidad de consumir varías hamburguesas. La primera puede ser muy útil si tengo hambre, depronto la segunda también, pero llega un punto de saturación. La utilidad no crece linealmente de forma irrestricta. En este modulo, aprenderemos técnicas para lidiar con situaciones similares donde la respuesta es no lineal en los predictores. En particular, será una introducción para correr modelos lineales generales en Python (GLM, por sus siglas en inglés).

In [ ]:
from __future__ import print_function

# para estructura de datos
import pandas as pd
import numpy as np

# para gráficas e interacciones
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sn
import plotly 
import plotnine as p9
from plotnine import ggplot, geom_point, geom_line, aes, geom_smooth, facet_wrap, themes
import bqplot as bq
from bqplot import pyplot as bqplt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import ColumnDataSource, Slider, Column, LinearInterpolator, CategoricalColorMapper, Legend, LegendItem
from bokeh.layouts import row, column
from bokeh.io import push_notebook
from bokeh.palettes import all_palettes, gray, inferno, magma, viridis, cividis, turbo
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


# para estadística 
import statsmodels.formula.api as smf
import statsmodels.api as sm 
#from statsmodels.sandbox.regression.gmm import IV2SLS 
from linearmodels.iv import IV2SLS
import scipy
from scipy.optimize import minimize
from scipy.stats import gaussian_kde
from pscore_match.match import Match, whichMatched




In [ ]:
# Data
# Cancer (https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/)
# Tumor information (e.g. size, radius, other measures)
cancer = pd.read_csv('DataSets/Cancer.csv')
cancer.insert(0, 'diagnosis_recode', 1) #0 malignant, 1 benign
cancer.loc[cancer['diagnosis'] == 'M','diagnosis_recode'] = 0


#Police arrests (Data Analysis Using Regression and Multilevel/Hierarchical Models de Andrew Gelman & Jennifer Hill)
Police = pd.read_csv("DataSets/GelmanHill_reg_hierarchical/frisk_with_noise.csv")
#precincts are numbered
#ethnicity (eth) 1=black, 2=hispanic, 3=white
#crime type 1=violent, 2=weapons, 3=property, 4=drug
Police = Police.drop(Police[Police['past.arrests'] == 0].index) #only one row removed
Police = Police.astype({'eth': 'category', 'precinct':'category'})

#Water wells (Arsenic)
wells = pd.read_stata('DataSets/GelmanHill_reg_hierarchical//arsenic/all.dta')
wells_itr = pd.read_stata('DataSets/GelmanHill_reg_hierarchical//arsenic/all.dta', iterator=True) #tiene, entre otras cosas, la descripción de las variables
wells_itr.variable_labels() #descripción de columnas


#Storable votes
storable = pd.read_csv("DataSets/GelmanHill_reg_hierarchical/storablevotes/6playergames.csv")
#the column value was randomly (uniform) assigned to subjects at the start of a round and before casting a vote for theme 1 


#Fishing mode
fishing = pd.read_stata("DataSets/mus15data.dta")
fishing_itr = pd.read_stata('DataSets/mus15data.dta', iterator=True) #tiene, entre otras cosas, la descripción de las variables
fishing_itr.variable_labels() #descripción de columnas


#HISP (Health Insurance Subsidy Program)
HISP = pd.read_stata('DataSets/HISP/evaluation.dta')
HISP_itr = pd.read_stata('DataSets/HISP/evaluation.dta', iterator=True) #tiene, entre otras cosas, la descripción de las variables
HISP_itr.variable_labels()


#M & M
#Fuente: https://joshmadison.com/2007/12/02/mms-color-distribution-analysis/
mandm = pd.read_csv("DataSets/mandm.csv")



## Visualización: distribuciones y regresiones

Una regresión OLS se puede escribir de la siguiente forma:

$$ Y \sim  Normal(link(\beta_0 + \beta_1X), \sigma^2)  $$

Link puede ser cualquier función. En el caso de una regresión tradicional, link es la identidad.

$$ Y \sim  Normal(\beta_0 + \beta_1X, \sigma^2)  $$

¿Podemos usar otras distribuciones diferentes a la normal? ¿Otras funciones link diferentes a identidad? Si, con generalized linear models. Antes de ver cómo se hace en Python, veamos esta idea de que las regresiones son distribuciones. 

Vamos a ver como sube o baja el promedio de la distribución normal, `Y`, a medida que cambia el valor de las variables independientes, `X`

In [ ]:
# prepare some data
nsamples = 500
noise = 500
betas = [4, 10.5, 0.5]
d = {'x1' : np.random.normal(10, 40, size = nsamples),
    'x2' : np.random.normal(1, 4, size = nsamples)}
d['y'] = betas[0] + betas[1]*d['x1'] + betas[2]*d['x2']
d['y_noise'] = d['y'] + np.random.normal(0, noise, size = nsamples)
dataT = pd.DataFrame(d)

model = smf.ols('y_noise ~ x1 + x2 ', data = dataT)
results = model.fit()
predictions = results.predict(dataT[['x1','x2']])
dataT['predictions'] = predictions
#print(results.summary())
source = ColumnDataSource(dataT) #Bokeh needs data in this format


# output to notebook inline or html file
output_notebook()
#output_file("lines.html") #creates a html file (can be shared)

## create scales for desired attributes
#color_scale = CategoricalColorMapper(factors=list(df.species.unique()), palette=all_palettes['Colorblind'])
size_scale = LinearInterpolator(x=[min(dataT['x2']), max(dataT['x2'])], y=[2, 8]) #y is the range of size of dots


# create new plots with a title and axis labels
p = figure(title="Regresión", x_axis_label='x1', y_axis_label='y',
           tools="crosshair,pan,reset,save,wheel_zoom",
           x_range=[-150, 150], y_range=[-2200, 2200],
           plot_height=300, plot_width=500)

# add renderers
dot1 = p.circle('x1', 'y_noise', 
      size = dict(field='x2', transform=size_scale), color = '#C0C0C0',
      source = source)
dot0 = p.circle('x1', 'predictions', 
             size = dict(field='x2', transform=size_scale), color = 'forestgreen',
             source = source)
plot_data  = ColumnDataSource({'x1': [0], 'x2': [0], 'y' : [100]}) #init plot.data
dot2 = p.circle('x1', 'y',
      size = 10, color = '#FF4500', alpha = 0, 
      source = plot_data) 

legend = Legend(items=[LegendItem(label="predicción (media)", renderers=[dot0]),
                       LegendItem(label="predicción (muestras)", renderers=[dot2])],
               location='top_left',
               label_text_font_size = '7pt')
p.add_layout(legend)


LINE_ARGS = dict(color="#3A5785", line_color=None)
vhist, vedges = np.histogram(dataT['y_noise'], density=True, bins=20) #vertical histogram
vzeros = np.zeros(len(vedges)-1)
vmax = max(vhist)*1.1
pv = figure(title = "Muestras Y", toolbar_location=None, plot_width=100, plot_height = p.plot_height, 
            x_range=(0, 1.7*vmax), y_range=p.y_range, min_border=10, y_axis_location="right")
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = np.pi/4
pv.background_fill_color = "#ffffff"
pv.xgrid.visible = False
pv.ygrid.visible = False
pv.xaxis.visible = False
pv.yaxis.visible = False

source_hist = ColumnDataSource({"left": np.repeat(0,len(vedges)-1), 
                                "bottom": vedges[:-1], "top": vedges[1:], "right": vhist})
vh = pv.quad(left='left', bottom='bottom', top='top', right='right', source = source_hist,
             color="white", line_color="#000000")
x = dataT['y_noise'].sort_values()
pdf = scipy.stats.norm.pdf
source_density = ColumnDataSource({'x': pdf(x, betas[0] + betas[1]*dataT['x1'].mean() + betas[2]*dataT['x2'].mean(), noise),
                                  'y': x})
vd = pv.line(x='x', y = 'y',
        source = source_density, line_color = '#FF4500', line_width = 2)


#Set widgets
sX1 = Slider(start = d['x1'].min(), end = d['x1'].max(), value=0, step=1, title="x1", width = 175)
sX2 = Slider(start = d['x2'].min(), end = d['x2'].max(), value=0, step=.5, title="x2", width = 175)


# Define interactivity functions
from time import sleep
def my_widget_input_handler(attr, old, new):
    x1 = np.repeat(sX1.value, nsamples)
    x2 = np.repeat(sX2.value, nsamples)
    y = betas[0] + betas[1]*x1 + betas[2]*x2  + np.random.normal(0, 0.75*noise, size = nsamples)
    
    plot_data.data = {'x1': x1, 'x2': x2, 'y': y}
    #dot2.glyph.fill_color = '#FF4500'
    dot2.glyph.fill_alpha = 1
    dot2.glyph.line_alpha = 1
    dot2.glyph.size = 0.5*abs(sX2.value) + 1
    
    vhist, vedges = np.histogram(y, density=True, bins=20) #vertical histogram
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1
    source_hist.data = {"left": np.repeat(0,len(vedges)-1), "bottom": vedges[:-1], 
                        "top": vedges[1:], "right": vhist}
    
    x = pd.Series(y).sort_values()
    source_density.data = {'x': pdf(x, betas[0] + betas[1]*x1 + betas[2]*x2, 0.75*noise), 
                           'y': x}


# Define widgets callbacks
sX1.on_change('value', my_widget_input_handler)
sX2.on_change('value', my_widget_input_handler)


# Show the plot
layout = row(p, pv, column(sX1,sX2))
def modify_doc(doc):
    doc.add_root(row(layout, width=50))
    doc.title = "Sliders"
    
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)



En la anterior visualización, vemos como la formula de la regresión es el promedio de una distribución normal. 

$$ Y \sim  Normal(\beta_0 + \beta_1X, \sigma^2)  $$

Las distribuciones normales pueden tomar valores positivos y negativos. Es apropiada, entonces, si la variable `Y` toma valores normales y lineales. Pero veamos un ejemplo que no es así: variables binarias (e.g. gano/perdio, vivo/muerto, 1/0, etc.).

In [ ]:
# NOTA: hay que correr la anterior celda
# prepare some data
nsamples = 1000 
def logistic(x):
    return 1 / (1 + np.exp(-x))
betas = [1, 2, 0.15]
d = {'x1' : np.random.normal(0, 5, size = nsamples),
     'x2' : np.random.normal(0, 5, size = nsamples)}
d['y'] = logistic(betas[0] + betas[1]*d['x1'] + betas[2]*d['x2']) #y son probabilidades!
d['y_noise'] = np.random.binomial(n = 1, p = d['y'])
dataT = pd.DataFrame(d)

model = smf.glm('y_noise ~ x1 + x2 ', data = dataT, 
                family = sm.families.Binomial())
results = model.fit()
predictions = results.predict(dataT[['x1','x2']])
dataT['predictions'] = predictions #Note como las predicciones de Y estan en [0,1]
#print(results.summary())
source = dataT #Bokeh needs data in this format

# output to notebook inline or html file
output_notebook()

## create scales for desired attributes
#color_scale = CategoricalColorMapper(factors=list(df.species.unique()), palette=all_palettes['Colorblind'])
size_scale = LinearInterpolator(x=[min(dataT['x2']), max(dataT['x2'])], y=[2, 10]) #y is the range of size of dots


# create new plots with a title and axis labels
p = figure(title="Regresión", x_axis_label='x1', y_axis_label='y',
           tools="crosshair,pan,reset,save,wheel_zoom",
           x_range=[-20, 20], y_range=[-0.1, 1.1],
           plot_height=300, plot_width=500)

# add renderers
circle1 = p.circle('x1', 'y_noise', 
                   size = dict(field='x2', transform=size_scale), color = '#C0C0C0',
                   source = source)
circle2 = p.circle('x1', 'predictions', 
                   color = 'forestgreen', alpha =0.5,
                   size = dict(field='x2', transform=size_scale),
                   source = source)

plot_data  = ColumnDataSource({'x1': [0], 'x2': [0], 'y' : [100]}) #init plot data
circle3 = p.circle('x1', 'y',
      size = 10, color = '#FF4500', alpha = 0, 
      source = plot_data) 

legend = Legend(items=[LegendItem(label="Data", renderers=[circle1]),
                       LegendItem(label="predicción (prob. que Y sea 1)", renderers=[circle2]),
                       LegendItem(label="predicción (muestras)", renderers=[circle3])],
               location='top_left',
               label_text_font_size = '7pt')
p.add_layout(legend)


LINE_ARGS = dict(color="#3A5785", line_color=None)
vhist, vedges = np.histogram(dataT['y_noise'], density=True, bins=20) #vertical histogram
vzeros = np.zeros(len(vedges)-1)
vmax = max(vhist)*1.1
pv = figure(title = "Muestras Y", toolbar_location=None, plot_width=100, plot_height = p.plot_height, 
            x_range=(0, 1.1), y_range=p.y_range, min_border=10, y_axis_location="right")
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = np.pi/4
pv.background_fill_color = "#ffffff"
pv.xgrid.visible = False
pv.ygrid.visible = False
pv.xaxis.visible = True
pv.yaxis.visible = False

source_hist = ColumnDataSource({"left": np.repeat(0,len(vedges)-1), 
                                "bottom": vedges[:-1], "top": vedges[1:], "right": vhist/vhist.sum()})
vh = pv.quad(left='left', bottom='bottom', top='top', right='right', source = source_hist,
             color="white", line_color="#000000")
x = dataT['y_noise'].sort_values()
pdf = scipy.stats.binom.pmf
n = 1
pr = logistic(betas[0] + betas[1]*dataT['x1'].mean() + betas[2]*dataT['x2'].mean())
source_density = ColumnDataSource({'x': pdf(x, n, pr),
                                   'y': x})
vd = pv.line(x='x', y = 'y', alpha = 0,
             source = source_density, line_color = '#FF4500', line_width = 2)


#Set widgets
sX1 = Slider(start = d['x1'].min(), end = d['x1'].max(), value=0, step=.01, title="x1", width = 175)
sX2 = Slider(start = d['x2'].min(), end = d['x2'].max(), value=0, step=.05, title="x2", width = 175)


# Define interactivity functions
from time import sleep
def my_widget_input_handler(attr, old, new):
    x1 = np.repeat(sX1.value, nsamples)
    x2 = np.repeat(sX2.value, nsamples)
    n = 1
    pr = logistic(betas[0] + betas[1]*x1 + betas[2]*x2)
    y =  np.random.binomial(n, pr)
    

    plot_data.data = {'x1': x1, 'x2': x2, 'y': y}
    #dot2.glyph.fill_color = '#FF4500'
    circle3.glyph.fill_alpha = 1
    circle3.glyph.line_alpha = 1
    circle3.glyph.size = 12
    vd.glyph.line_alpha = 1
    
    vhist, vedges = np.histogram(y, density=True, bins=20) #vertical histogram
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1
    source_hist.data = {"left": np.repeat(0,len(vedges)-1), "bottom": vedges[:-1], 
                        "top": vedges[1:], "right": vhist/vhist.sum()}
    
    x = pd.Series(y).sort_values()
    source_density.data = {'x': pdf(x, n, pr), 
                           'y': x}


# Define widgets callbacks
sX1.on_change('value', my_widget_input_handler)
sX2.on_change('value', my_widget_input_handler)


# Show the plot


# Show the plot
layout = row(p, pv, column(sX1,sX2))
#layout = row(p)    
handler2 = FunctionHandler(modify_doc)
app2 = Application(handler2)
show(app2)


## Efectos marginales no lineales dependen del valor del predictor

Una diferencia importante entre modelos lineales y no lineales es el cuidado que hay que tener cuando se interpretan los coeficientes. En un modelo lineal los $\beta$ son efectos marginales/derivadas i.e. cuanto cambia `Y` con un cambio en `X` (i.e. la primera derivada es una constante). En un modelo no lineal es más complicado, la derivada depende de la función no lineal, y puede ocurrir que los efectos marginales/derivadas varian con el nivel de `X` (i.e. la primera derivada podría incluir `X`). En términos generales, con regresiones no lineales es mejor siempre interpretar los coeficientes **dado** un `X`. 

In [ ]:
# Veamos dos regresiones, una lineal y otra no-lineal, con la misma data 
# Primero veamos los datos
fig, ax = plt.subplots(figsize=(5,5))
sn.boxplot(x="diagnosis", y="radius_mean", data=cancer, ax = ax) 
#sn.swarmplot(x="diagnosis", y="radius_mean", data=cancer, color = 'black', alpha=0.2, ax = ax)
ax.set_ylabel('Tamaño del tumor (radio)')
ax.set_xlabel('M: maligno, B: benigno')
ax.set_title('Box plot')


Parece que los tumores malignos son más grandes. No es sorpresivo pero confirmemoslo con una regresión.

In [ ]:
# Regresión lineal OLS
model = smf.ols('diagnosis_recode ~ radius_mean', data = cancer)
results = model.fit()
print("El efecto marginal es: " + str(round(results.params[1],2))) #Tumores benignos tienen un radio más pequeño (p<0.05) 


El tamaño del tumor afecta que se codifique como maligno o no, pero la pendiente y el intercepto son de una línea. Ese -0.1 no es cambio en la probabilidad de ser un tumor maligno; es una pendiente. 

Ahora tratemos una regresión logística dado que el diagnóstico es binario, maligno o benigno, y con esta regresión podemos obtener probabilidades.

Nos interesa el cambio de probabilidad de ser maligno con un aumento en el tamaño. Recordar que en una regresión logística, la formula de la regresión es igual al logit:

$$ 
\begin{equation} 
x_j\beta_j = log\left(\frac{p}{1-p}\right) = log(p) - log(1-p)
\end{equation}
$$

La primera derivada, que nos daría el efecto marginal de interés, va igual depender de x pues p lo obtenemos de la función logística. Esta es la primera derivada (ver https://stats.stackexchange.com/questions/19764/marginal-effect-of-probit-and-logit-model):

$$ efecto \; marginal \; de \; j = \beta_jp(1-p)$$

Donde p es:

$$ p = \frac{1}{1+exp^{-x\beta}}$$

Por ello hay que decidir en cuál x se evalua el efecto marginal. Tal vez el más directo es en el promedio de `X`.


In [ ]:
# Regresión logistica GLM 
model = smf.glm('diagnosis_recode ~ radius_mean', data = cancer, 
                family = sm.families.Binomial())
results = model.fit()
xBeta = results.params[0]+results.params[1]*cancer['radius_mean'].mean() # punto a evaluar
p = 1/(1+np.exp(-xBeta))
em = results.params[1]*p*(1-p) # efecto marginal
print("Una reducción en tamaño disminuye la probabilidad de ser maligno en " + str(round(em,2)) + " (en el promedio).")
print("El tamaño afecta la probabilidad relativa de ser maligno relativo a benigno, la cual es " + 
      str(round(1/np.exp(results.params[1]),2)) + " (odds ratio)")


## Diferentes sentidos de likelihood

In [ ]:
# data artificial 
nobs=100
mean = 400
stddev = 200
data_dgp = np.random.normal(mean, stddev, nobs)
fig, ax = plt.subplots(figsize = (5,5))
sn.scatterplot(x = np.arange(len(data_dgp)), y = data_dgp, ax = ax)


# likelihood es la probabilidad de los datos
hipotesis_mean = 200 #asumamos que sabemos sd
den = scipy.stats.norm.pdf(data_dgp, hipotesis_mean, stddev)
l = np.prod(den) # likelihood de la data (underflow, se va a cero rápidamente)
print('Este es el likelihood: ' + str(round(l,3)))
ll = np.log(den).sum() #log likelihood de la data. En logaritmos, la multiplicación se vuelve suma y no hay underflow.
print('Este es el log likelihood: ' + str(round(ll,3)))


El likelihood es la probabilidad de la data dada la hipótesis. ¿Es una buena hipótesis? ¿buen log likelihood? Hay que probar más hipótesis. Usaremos el log likelihood para evitar underflow.

In [ ]:
# No confundir likelihood con MLE que algunas veces se visualiza como sigue 
# Probemos varios promedios para ver cual es mejor 
hipotesis_mean = np.linspace(-275,1000, 100)
ll = [] #log likelihoods
for hypothesis in hipotesis_mean:
    den = scipy.stats.norm.pdf(data_dgp, hypothesis, stddev)
    ll.append(np.log(den).sum())
ll = np.array(ll)
fig, ax = plt.subplots(figsize = (5,5))
sn.lineplot(x = hipotesis_mean, y = ll, ax = ax)
ax.set_ylabel('log. likelihood')
ax.set_xlabel('hipótesis')
ax.set_title('MLE: p(parametros|data)')
idx = ll == max(ll)
ax.scatter(hipotesis_mean[idx], ll[idx], s = 100, c = 'red', label = 'MLE')
ax.legend()

# La hipótesis que maximiza el likelihood (MLE) es la que definimos antes 

## Maximum likelihood estimation (MLE)

El MLE es una alternativa a mínimos cuadrados ordinarios (OLS). Para obtener el estimativo MLE muchas veces (sino la mayoria) tenemos que usar métodos numéricos y estrategias computacionales que nos permiten buscar maximizar la probabilidad de los datos entre varias posibles hipótesis de parametros. En regresiones lineales simples habiamos usado OLS. Hagamoslo con MLE.

In [ ]:
# linear
# Lo primero es definir una función que nos diga cómo calcular el (log) likelihood
def ll(params):
    # log likelihood (ll)
    # Esta función es la que vamos a pasar a un algoritmo
    # para que proponga parametros (params) que minimice
    # el log. likelihood negativo
    
    intercept = params[0]
    beta1 = params[1]
    sigma = params[2]
    
    mu = intercept + beta1*x
    l = scipy.stats.norm.pdf(y, mu, sigma) #likelihood
    l[l==0] = 1**(-300) #Hack to avoid infinity (beware: only for pedagogical purposes)
    ll = -np.log(l).sum() #negative log likelihood (el algoritmo va a minimizar)
    return ll 

# Parametros para iniciar el algoritmo
guess = np.array([0,0,0.1])

# info global que usa la funcion likelihood
x = cancer['radius_mean']
y = cancer['diagnosis_recode'] 

optim = minimize(ll, guess, method = 'L-BFGS-B',
                 bounds= ((-10, 10), (-10, 10), (0,10)), #a tuple for each parameter (as ordered in the likelihood function)
                 options={'disp': True})
results_mle = pd.DataFrame({'coef':optim['x']})
results_mle.index=['Intercept','beta1','sigma']   
model = smf.ols('diagnosis_recode~radius_mean', data = cancer)
results_ols = model.fit()
print('RESULTADOS MLE')
print(np.round(results_mle.head(3), 4)) 
print('\n')
print('RESULTADOS OLS')
print(results_ols.summary()) 

#Compare los resultados con MLE y OLS. Deben ser iguales.

## Regresión no lineal: Poisson

Otra regresion para modelar conteos es usando una distribución discreta como la Poisson

$$ y \sim Poisson(\theta)$$

Donde

$$ \theta = e^{x\beta}$$

Veamos un ejemplo con una data que tiene el conteo de cuantas veces paran a una persona

In [ ]:
#Poisson police stops ####
# stops es cuantas veces la policia para a una persona para requisar
# constant term
# ¿Por que usamos para offset past.arrests? baseline
model_null = smf.glm('stops ~ 1', offset=np.log(Police['past.arrests']), 
                family = sm.families.Poisson(), data = Police)
results_null = model_null.fit(cov_type='HC3')
print(results_null.summary())


# ethnicity indicator eth 1=black, 2=hispanic, 3=white.
model_A = smf.glm('stops ~ eth', offset = np.log(Police['past.arrests']), 
                family = sm.families.Poisson(), data = Police)
results_A = model_A.fit(cov_type='HC3')
print(results_A.summary())


# ethnicity & precints indicators
model_B = smf.glm('stops ~ eth + precinct', offset = np.log(Police['past.arrests']), 
                family = sm.families.Poisson(), data = Police)
results_B = model_B.fit(cov_type='HC3')
print(results_B.summary())


### Ejercicio de interpretación

Modelo A:

* Interpretar coeficientes exp(coef).  eth 1=black,	2=hispanic,3=white (recordar: es relativo al baseline i.e. la categoria que no aparece eth:1:black). ¿A quíen arrestan más que los negros? ¿Hispanos o blancos?.
* Comparar modelo A y modelo null. Use el AIC (método .aic de las variables `results`) (menor es mejor). ¿vale la pena meter un regresor de etnicidad?

Modelo B:

* Interpretar coeficientes de etnicidad. 
* ¿Por qué blancos ahora si es significativo?
* ¿Por qué ahora es más negativo el coeficiente para blancos eth:3?
* Comparar AIC con modelo A que solo tiene etnicidad (menor es mejor). ¿Vale la pena meter el precinto?




## Propensity Score Matching (PSM)

Si usamos probit o logit (que es usual), el propensity score es la probabilidad de estar en un grupo (en python, si el fit está en results, entonces el score es results.predict). Si los grupos son tratamiento y control, queremos que ambos tengan scores similares i.e. que la probabilidad de ser tratado sea parecida, dado los covariates (e.g. demográficos). En otras palabras, que ambos grupos sean lo más parecidos posibles. Los algoritmos de PSM usan alguna regla para decidir qué es un score cercano (e.g. k nearest neighbor).

In [ ]:
# pscore-match documentation (http://www.kellieottoboni.com/pscore_match/index.html) 
idx = HISP['round']==0 #para hacer matching con características antes del tratamiento
HISP_baseline = HISP.loc[idx].copy() 
#clasificar households en funcion de unas caracteristicas
treatment = np.array(HISP_baseline['enrolled'])
model = smf.glm('enrolled ~ age_hh + age_sp + educ_hh + educ_sp +'\
                'female_hh + indigenous + hhsize + dirtfloor + bathroom +'\
                'land + hospital_distance', 
                data = HISP_baseline, 
                family = sm.families.Binomial())
results = model.fit()
pscore = results.predict()
pairs = Match(treatment, pscore)
#many-to one: one control individual is matched to many treatment individuals
#one-to one: one control individual is matched to one treatment individual
##si no sirve, cambiar el codigo de whichMatched de data.ix a data.iloc
pairs.create(method='many-to-one', many_method='knn', k=5, replace=True) 
#Otros metodos:
# ‘one-to-one’ or ‘many-to-one’
# many_method: (solo para many-to-one)
#       "caliper" (default) to select all matches within a given range, "knn" for k nearest neighbors,  
HISP_baseline.insert(0,'pscore',pscore) 
HISP_baseline.insert(0,'treatment',treatment) 
HISP_matched = whichMatched(pairs, HISP_baseline) 

En la base HISP_matched, los grupos tratamiento y control son lo más parecido posibles dado los demográficos que definimos en la lista de covariables. Veamos como se veian los propensity scores antes y después.

In [ ]:
plt.figure(1)
plt.subplot(121)
density0 = gaussian_kde(pscore[treatment==0])
density1 = gaussian_kde(pscore[treatment==1])
xs = np.linspace(0,1,200)
plt.plot(xs,density0(xs),color='black', label = 'control')
plt.fill_between(xs,density1(xs),color='gray', label = 'tratamiento')
plt.legend()
#plt.text(0.03, 35, 'Control Group')
#plt.text(0.06, 10, 'Treatment Group')
plt.title('Before Matching')
plt.axis([0,1,0,3.5])
plt.xlabel('Propensity Score')
plt.ylabel('Density')

plt.subplot(122)
density0_post = gaussian_kde(HISP_matched.pscore[HISP_matched.treatment==0])
density1_post = gaussian_kde(HISP_matched.pscore[HISP_matched.treatment==1])
xs = np.linspace(0,1,200)
plt.plot(xs,density0_post(xs),color='black')
plt.fill_between(xs,density1_post(xs),color='gray')
plt.title('After Matching')
plt.axis([0,1,0,3.5])
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.tight_layout()
plt.show()

Note como después de hacer el PSM, la probabilidad de estar en el grupo de control o tratamiento es bien parecida: las distribuciones del propensity score se sobrelapan. 

Hicimos matching con la data en el momento 0, antes de hacer el tratamiento. Para hacer análisis como Diff-In-Diff, tenemos que traer la información del momento 1, después de implementar el tratamiento. En la base de datos HISP tenemos identificadores (household_identifier)

In [ ]:
# Este loop toma tiempo, 30-60 segundos
for idx, hhi in enumerate(HISP_matched['household_identifier']):
    idx2 = (HISP['household_identifier'] == hhi) & (HISP['round'] == 1)
    infoTemp = HISP.loc[idx2,:].copy()
    infoTemp.insert(0,'pscore', float('nan')) 
    infoTemp.insert(0,'treatment', float('nan')) 
    HISP_matched = pd.concat([HISP_matched, infoTemp])

Ahora podemos hacer nuestros análisis con la nueva base pareada por PSM

In [ ]:
#Podemos hacer diferencia en diferencias con la nueva base de datos
model = smf.ols("health_expenditures~round*enrolled + age_hh + age_sp +"\
           "educ_hh + educ_sp + female_hh + indigenous + hhsize +"\
           "dirtfloor + bathroom + land + hospital_distance",
         data = HISP_matched)
results = model.fit()
print(results.summary()) #La interaccion es el diff in diff 
print('\n')

idx0 = (HISP['enrolled'] == 0) & (HISP['round'] == 1)
idx1 = (HISP['enrolled'] == 1) & (HISP['round'] == 1)
eff = HISP.loc[idx0,'health_expenditures'].mean() - HISP.loc[idx1,'health_expenditures'].mean()
print("Mayor gastos de bolsillo en salud en control: " + str(round(eff,2)))


idx0 = (HISP_matched['enrolled'] == 0) & (HISP_matched['round'] == 1)
idx1 = (HISP_matched['enrolled'] == 1) & (HISP_matched['round'] == 1)
eff = HISP_matched.loc[idx0,'health_expenditures'].mean() - HISP_matched.loc[idx1,'health_expenditures'].mean()
print("Mayor gastos de bolsillo en salud en control (muestra PSM): " + str(round(eff,2)))
print('\n')
print("La política de enroll, haciendo match, parece menos fuerte, evitamos un sesgo.")


### Ejercicio - Hacer propensity score matching (PSM)

Vamos a analizar una base de datos que tiene información de pacientes vivos y muertos (http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/Crhc.html). Nos interesa saber el efecto de hacer un cateterismo al lado derecho del corazon. Haga lo siguiente:

1. Cargue la base de datos `corazon.csv` (en la carpeta DataSets). Pongala en una variable llamada `corazon`.
2. La variable `swang1` y `death` no son números. Recodifique la variable `swang1` para que RHC sea 1, y NO RHC sea 0. Haga lo mismo para la variable `death`.
3. Haga una regresión logística con un generalized lineal model (Tip: use el método smf.glm). Obtenga el propensity score (Tip: use el método .predict del objeto .fit). 
    - Variable endógena: `death`
    - Variable de interés: `swang1` (1: se hizo un cateterismo; 0: no se hizo)
    - Variables de control: `age`, `sex`, `race`, `edu`, `income`, `ninsclas` (tipo de seguro),`renal` (problemas renales), `neuro` (problemas neurológicos).
4. Haga una nueva base con grupos pareados por las variables de control del punto anterior. La variable de tratamiento es `death`. 
5. Grafique el propensity score antes y después del match (use el código donde hicimos esa gráfica para la base HISP). ¿Hay un buen match?
6. Haga la misma regresión del punto 3 con la base pareada. ¿Hay diferencias?

